# Generate trades using dedicated service - F#

### Overview
Retrieving trades over a look back period is a valuable indicator to highlight market activity. It can be correlated with a set of technical indicators to better analyze market movements and build a decision making parameter.

This sample is designed to request trades and illustrate market activity over a look back period.

This sample demonstrates how to request and plot from a dedicated data service **on-demand** trades over the time.

### Inputs/outputs
Trades extraction sample requires instrument's identifier, date time intervals as per inputs and returns the occured trades information.

### Services used
This sample uses *gRPC requests* in order to retrieve trades information from the hosted service. The queried endpoint in this script are:
* *TickTradesService*: to directly retrieve trades data from the server.

### Packages required
1. Systemathics:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.Tick.V1*
2. Open source:
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    * *XPlot.Plotly* as per display package
    
***

# Run Trades sample

### Step 1: Install packages

In [ ]:
#i "nuget: file:///home/jovyan/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis"
#r "nuget: XPlot.Plotly.Interactive"
#r "nuget: TaskBuilder.fs"

In [ ]:
open Systemathics.Apis.Helpers
open Systemathics.Apis.Type.Shared.V1
open Systemathics.Apis.Services.Tick.V1
open Google.Protobuf.WellKnownTypes
open Google.Type
open Grpc.Net.Client
open Grpc.Core
open XPlot.Plotly
open XPlot.Plotly.Interactive

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Get token as metadata
let headers = TokenHelpers.GetTokenAsMetaData();

// Create communication channel
let channel = ChannelHelpers.GetChannel();

### Step 3: Create and process request
To request *tick trades* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Tick trades request parameters

#### 3.1 Instrument selection

In [ ]:
let ticker = "AAPL"
let exchange = "BATS"

#### 3.2 Time period delimitation

In [ ]:
// Create time intervals
let start = new DateTime(2021, 03, 05)
let end' = new DateTime(2021, 03, 05)
let dateIntervals = 
  new DateInterval(
    StartDate = new Date ( Year = start.Year, Month = start.Month, Day = start.Day ),
    EndDate = new Date ( Year = end'.Year, Month = end'.Month, Day = end'.Day )
  )

// Build the tick trades request time interval (we are using Google date time format)
// UTC time zone
let timeInterval = 
  new TimeInterval(
    StartTime = new TimeOfDay ( Hours = 12, Minutes = 00, Seconds = 00 ),
    EndTime = new TimeOfDay ( Hours = 20, Minutes = 30, Seconds = 00 ) 
  )

#### 3.3 Request creation
The following code snippet creates *gRPC client*, process request and returns the request reply:

In [ ]:
// Generate constraints based on the previous time selection
let constraints = new Constraints() 
constraints.DateIntervals.Add(dateIntervals)
constraints.TimeIntervals.Add(timeInterval)

The following code snippets create requests for *trades* and instantiate the service:

In [ ]:
// Generate the tick trades request
let request = new TickTradesRequest ( Constraints = constraints )
let identifier = new Identifier ( Exchange = exchange, Ticker = ticker )
request.Identifiers.Add( [| identifier |]) 

In [ ]:
// Instantiate the tick trades service
let service = new TickTradesService.TickTradesServiceClient(channel)

### Step 4: Visualize data

#### 4.1 Retrieve trades data

The following code snippets retrieves trades data points:

In [ ]:
// Get the trades
open FSharp.Control.Tasks.V1
open System.Collections.Generic

let trades = List<Trade>()

let isNotNull obj =
    obj |> isNull |> not

let iterAsync (t: IAsyncEnumerable<'T>) (f : 'T -> unit) =
    let task = 
        task {
            let e = t.GetAsyncEnumerator()
            let mutable go = true
            try
                let! step = e.MoveNextAsync()
                go <- step
                while go do 
                    f e.Current
                    let! step = e.MoveNextAsync()
                    go <- step
            finally
                e.DisposeAsync().AsTask().Wait()
        }
    task.Wait()

let add (ttr : TickTradesResponse) =
    if(ttr |> isNotNull && ttr.Trade |> isNotNull) then
        trades.Add ttr.Trade

let call = service.TickTrades(request, headers)
let e = call.ResponseStream.ReadAllAsync()

iterAsync e add

The following code snippet displays request results:

In [ ]:
trades
|> Seq.length
|> display

#### 4.2 Plot the trade price
Display trades data over the look back period with the package of your choice.<br><i>`XPlot.Plotly`</i> is used as per open source display package.
Plot a simple line graph with the trade prices and volumes as follows: 

In [ ]:
let tradesGraph = 
    [
        new Scattergl(
            name = "Price",
            mode = "lines",
            x = (trades |> Seq.map (fun elt -> elt.TimeStamp.ToDateTime())),
            y = (trades |> Seq.map (fun elt -> elt.Price))
        )
        new Scattergl(
            name = "Size",
            mode = "lines",
            x = (trades |> Seq.map (fun elt -> elt.TimeStamp.ToDateTime())),
            y = (trades |> Seq.map (fun elt -> elt.Size)),
            yaxis = "y2"
        )
    ]

let layout = 
    new Layout.Layout( 
        title = $"Trades for {ticker}",
        xaxis = new Xaxis(title = "Time"),
        yaxis = new Yaxis (
            title = "Price", 
            autorange = true,
            side = "left"
        ),
        yaxis2 = new Yaxis (
            title = "Size", 
            autorange = true,
            side = "right"  
        )
    )

Chart.Plot(tradesGraph, layout)
|> Chart.WithWidth 1500
|> Chart.WithHeight 800
|> display